In [1]:
import role_permissions as rp
import pandas as pd
import pm4py

from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.conversion.log import converter as log_converter

In [2]:
# Data Import:
xes_path = "/Users/moritz_hawener/Documents/Work/Studium/Master/WS25/BPI/BPI Challenge 2017_1_all/BPI Challenge 2017.xes.gz"

# Read the XES log
event_log = xes_importer.apply(xes_path)

/opt/miniconda3/envs/bpi/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 31509/31509 [00:32<00:00, 975.11it/s] 


In [3]:
# Transform the event log to a pandas DataFrame
df = log_converter.apply(event_log, variant=log_converter.Variants.TO_DATA_FRAME)

In [4]:
# 1. Roles & Clustering
activity_clusters, resource_allocation_df, Z = rp.discover_roles(df)

# 2. Resource Assignment
resource_groups = rp.assign_resources_to_clusters(activity_clusters, resource_allocation_df, df)

Discovering organizational roles...
Assigning resources to clusters...


In [5]:
# Role Names Mapping (Could be externalized)
role_names = {
    1: "Core Process Handler",
    2: "Completion Specialist", 
    3: "Submission Handler",
    4: "Support & Communication Specialist",
    5: "Collections Specialist"
}

In [6]:
# 4. Context Definition
df = rp.define_execution_contexts(df, activity_clusters, resource_groups)

Defining execution contexts...


In [7]:
# 5. Capabilities
capabilities = rp.calculate_capabilities(df, resource_groups, role_names)

Calculating capabilities...


In [8]:
# 6. Build Model
organizational_model = rp.build_organizational_model(resource_groups, capabilities)

In [9]:
# 7. Generate simple permission map for backward compatibility
simple_permission_map = {}

for activity, cluster_id in activity_clusters.items():
    simple_permission_map[activity] = resource_groups[cluster_id]


In [10]:
# 7b. Context-aware permission map (OrdinoR - THE IMPORTANT ONE!)
context_permission_map = rp.generate_context_permission_map(
    df,
    organizational_model, 
    activity_clusters
)

Generating context permission map...


In [11]:
context_permission_map

defaultdict(list,
            {('A_Create Application', 'Very Complex', 'Morning'): ['User_1',
              'User_17',
              'User_52',
              'User_11',
              'User_117',
              'User_2',
              'User_38',
              'User_119',
              'User_30',
              'User_68',
              'User_19',
              'User_116',
              'User_101',
              'User_34',
              'User_31',
              'User_90',
              'User_89',
              'User_15',
              'User_16',
              'User_12',
              'User_3',
              'User_102',
              'User_114',
              'User_118',
              'User_95',
              'User_8',
              'User_113',
              'User_100',
              'User_28',
              'User_27',
              'User_99',
              'User_112',
              'User_75',
              'User_21',
              'User_87',
              'User_20',
              'User_43'

In [12]:
# 8. Save
rp.save_output(simple_permission_map, activity_clusters, resource_groups, 
                role_names, organizational_model, context_permission_map)
print("Process complete.")

Saving output files...
✅ Saved organizational_model.pkl
✅ Saved organizational_model.json
✅ Saved context_permissions.csv (401 contexts)
✅ Saved role_summary.csv

📦 Export Summary:
   Simple permissions: 26 activities
   Context permissions: 401 contexts
   Resource groups: 5 clusters
   Total resources: 149 resources
Process complete.
